In [50]:
import pandas as pd
import numpy as np
import glob
import geopandas
import geopandas.tools
from shapely.geometry import Point
import datetime as dt

In [35]:
path = r'C:\Study\Hackathon\2014-2016 311 Data' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

In [36]:
frame.shape

(1165419, 36)

In [37]:
list(frame)

['SERVICEREQUESTID',
 'SERVICEPRIORITY',
 'SERVICECODE',
 'SERVICECODEDESCRIPTION',
 'SERVICETYPECODE',
 'SERVICETYPECODEDESCRIPTION',
 'SERVICEORDERDATE',
 'SERVICEORDERSTATUS',
 'SERVICECALLCOUNT',
 'AGENCYABBREVIATION',
 'INSPECTIONFLAG',
 'INSPECTIONDATE',
 'RESOLUTION',
 'RESOLUTIONDATE',
 'SERVICEDUEDATE',
 'SERVICENOTES',
 'PARENTSERVICEREQUESTID',
 'ADDDATE',
 'LASTMODIFIEDDATE',
 'SITEADDRESS',
 'LATITUDE',
 'LONGITUDE',
 'ZIPCODE',
 'MARADDRESSREPOSITORYID',
 'DCSTATADDRESSKEY',
 'DCSTATLOCATIONKEY',
 'WARD',
 'ANC',
 'SMD',
 'DISTRICT',
 'PSA',
 'NEIGHBORHOODCLUSTER',
 'HOTSPOT2006NAME',
 'HOTSPOT2005NAME',
 'HOTSPOT2004NAME',
 'SERVICESOURCECODE']

In [38]:
frame_nonRodent = frame[frame['SERVICECODE'] != 'S0311']

In [39]:
frame_nonRodent.shape

(1157315, 36)

In [40]:
frame_nonRodent['LATITUDE'].isnull().sum()

222700

In [41]:
# Removing records with NULL values in lat-long - to get the census code block piece to run error-free
frame_nonRodent = frame_nonRodent[np.isfinite(frame_nonRodent['LATITUDE']) & np.isfinite(frame_nonRodent['LONGITUDE']) ]
frame_nonRodent.shape

(934615, 36)

In [42]:
column_names = list(frame_nonRodent.columns.values)
 
frame_nonRodent['geometry'] = frame_nonRodent.apply(lambda row: Point(row['LONGITUDE'],row['LATITUDE']), axis=1)
frame_nonRodent = geopandas.GeoDataFrame(frame_nonRodent, geometry='geometry')
frame_nonRodent.crs = {'init': 'epsg:4326'}
 
census_blocks = geopandas.GeoDataFrame.from_file('dc_2010_block_shapefiles/tl_2016_11_tabblock10.shp')
census_blocks.crs = {'init': 'epsg:4326'}
 
# result = geopandas.tools.sjoin(frame_nonRodent, census_blocks[['BLOCKCE10', 'geometry']], how='inner')
result = geopandas.tools.sjoin(frame_nonRodent[['geometry']], census_blocks[['BLOCKCE10', 'geometry']], how='left')
 
frame_nonRodent['census_block'] = result['BLOCKCE10']
frame_nonRodent = frame_nonRodent[column_names + ['census_block']]
 

In [55]:
frame_nonRodent.shape
list(frame_nonRodent)

['SERVICEREQUESTID',
 'SERVICEPRIORITY',
 'SERVICECODE',
 'SERVICECODEDESCRIPTION',
 'SERVICETYPECODE',
 'SERVICETYPECODEDESCRIPTION',
 'SERVICEORDERDATE',
 'SERVICEORDERSTATUS',
 'SERVICECALLCOUNT',
 'AGENCYABBREVIATION',
 'INSPECTIONFLAG',
 'INSPECTIONDATE',
 'RESOLUTION',
 'RESOLUTIONDATE',
 'SERVICEDUEDATE',
 'SERVICENOTES',
 'PARENTSERVICEREQUESTID',
 'ADDDATE',
 'LASTMODIFIEDDATE',
 'SITEADDRESS',
 'LATITUDE',
 'LONGITUDE',
 'ZIPCODE',
 'MARADDRESSREPOSITORYID',
 'DCSTATADDRESSKEY',
 'DCSTATLOCATIONKEY',
 'WARD',
 'ANC',
 'SMD',
 'DISTRICT',
 'PSA',
 'NEIGHBORHOODCLUSTER',
 'HOTSPOT2006NAME',
 'HOTSPOT2005NAME',
 'HOTSPOT2004NAME',
 'SERVICESOURCECODE',
 'census_block',
 'year',
 'week']

In [86]:
frame_nonRodent['SERVICEORDERDATE'] = pd.to_datetime(frame_nonRodent['SERVICEORDERDATE'])
frame_nonRodent['year'] = frame_nonRodent['SERVICEORDERDATE'].dt.year
frame_nonRodent['week'] = frame_nonRodent['SERVICEORDERDATE'].dt.week

In [61]:
df = frame_nonRodent.groupby(['SERVICECODE','census_block', 'year', 'week']).size().reset_index(name='value')

In [62]:
list(df)

['SERVICECODE', 'census_block', 'year', 'week', 'value']

In [66]:
df = df.rename(columns={'SERVICECODE': 'feature_type', 'census_block': 'census_block_2010'})
df['feature_id'] = "311_service_requests"
df['feature_subtype'] = ""
df.shape

(290900, 7)

In [67]:
list(df)

['feature_type',
 'census_block_2010',
 'year',
 'week',
 'value',
 'feature_id',
 'feature_subtype']

In [69]:
cols = ['feature_id', 'feature_type', 'feature_subtype', 'year', 'week','census_block_2010','value']
df = df[cols]

In [70]:
list(df)

['feature_id',
 'feature_type',
 'feature_subtype',
 'year',
 'week',
 'census_block_2010',
 'value']

In [71]:
df.to_csv('DataIssue_8.csv', index=False)